In [102]:
import pandas as pd


In [103]:
df=pd.read_csv('medical_data.csv')

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   death           580 non-null    int64  
 1   amiodarone      580 non-null    int64  
 2   loop_diuretics  580 non-null    int64  
 3   ivabradine      580 non-null    int64  
 4   ARB             580 non-null    int64  
 5   digoxin         580 non-null    int64  
 6   MRA             580 non-null    int64  
 7   heart_failure   580 non-null    int64  
 8   AOS             580 non-null    int64  
 9   SBP             580 non-null    float64
 10  DBP             580 non-null    float64
 11  PLT             580 non-null    float64
 12  LDL             580 non-null    float64
 13  HDL             580 non-null    float64
 14  LVEF            580 non-null    float64
 15  Na+             580 non-null    float64
 16  K+              580 non-null    float64
 17  MPV             580 non-null    flo

In [105]:
df.columns

Index(['death', 'amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin',
       'MRA', 'heart_failure', 'AOS', 'SBP', 'DBP', 'PLT', 'LDL', 'HDL',
       'LVEF', 'Na+', 'K+', 'MPV'],
      dtype='object')

In [106]:
df.head()

,death,amiodarone,loop_diuretics,ivabradine,ARB,digoxin,MRA,heart_failure,AOS,SBP,DBP,PLT,LDL,HDL,LVEF,Na+,K+,MPV
0,0,0,0,1,1,1,0,2,0,62.0,126.0,196.0,50.0,42.0,28.0,136.0,4.30,9.9
1,0,1,1,0,1,0,1,0,1,72.0,108.0,245.0,59.0,85.0,25.0,147.0,4.58,13.3
2,0,0,0,0,0,1,1,2,2,73.0,109.0,219.0,79.0,61.0,14.0,133.0,4.05,1.5
3,0,0,0,1,1,1,0,2,1,55.0,114.0,294.0,97.0,55.0,8.0,150.0,5.34,7.8
4,0,0,1,0,1,1,1,2,2,70.0,95.0,293.0,96.0,30.0,28.0,151.0,5.25,6.8


In [107]:
#cat_vars = ['amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin', 'MRA', 'heart_failure', 'AOS']
#cat_vars = [col for col in df.columns if df[col].dtype == 'object']
#numerical_vars = [col for col in df.columns if df[col].dtype != 'object']

In [108]:
#num_vars=['SBP', 'DBP', 'PLT', 'LDL', 'HDL','LVEF', 'Na+', 'K+', 'MPV']

In [109]:
#chi_tuple_list = [(var,pval ) for var,pval in chi_result]


In [110]:
cat_vars=[]
numerical_vars=[]
for col in df.columns:
    if df[col].dtype == 'int64':
        # categorical variable encoded as 0 or 1
        if(col!="death"):
            cat_vars.append(col)
    else:
        # numerical variable
        numerical_vars.append(col)
print(len(cat_vars),len(numerical_vars))
print("num",numerical_vars)
print("cat",cat_vars)

8 9
num ['SBP', 'DBP', 'PLT', 'LDL', 'HDL', 'LVEF', 'Na+', 'K+', 'MPV']
cat ['amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin', 'MRA', 'heart_failure', 'AOS']


In [111]:
from scipy.stats import chi2_contingency
chi_result=[]
for var in cat_vars:
    contingency_table = pd.crosstab(df['death'], df[var])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    chi_result.append((f"{var,p_value}"))
print(chi_result)

["('amiodarone', 3.37882118161796e-15)", "('loop_diuretics', 1.154302903307273e-22)", "('ivabradine', 0.014373704380245855)", "('ARB', 0.2337835245845317)", "('digoxin', 0.5185149615843472)", "('MRA', 0.28840961222458317)", "('heart_failure', 7.608379512530399e-12)", "('AOS', 0.9740161217622956)"]


In [112]:
import pandas as pd
from scipy.stats import shapiro, ttest_ind, mannwhitneyu

# Split data by death variable
death_data = df[df['death'] == 1]
alive_data = df[df['death'] == 0]

#cat_vars = ['amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin', 'MRA', 'heart_failure', 'AOS']
chi_result=[]
for var in cat_vars:
    contingency_table = pd.crosstab(df['death'], df[var])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    #chi_tuple=[(var,round(p_value,4))]
    chi_result.append((var,round(p_value,4)))


# Create an empty dataframe to store the test results
test_results = pd.DataFrame(columns=['Variable', 'Test', 'Statistic', 'p-value'])
shapiro_test_results=[]
mann_test_results=[]
ttest_results=[]
# Loop through each numerical variable
#numerical_vars = ['SBP', 'DBP', 'PLT', 'LDL', 'HDL', 'LVEF', 'Na+', 'K+', 'MPV']
for var in numerical_vars:
    # Check normality of each sample
    death_stat, death_pval = shapiro(death_data[var])
    alive_stat, alive_pval = shapiro(alive_data[var])
    shapiro_test_results.append((var, (round(alive_pval,4), round(death_pval,4))))
    
    # Perform t-test or Mann-Whitney U test based on normality test results
    if death_pval > 0.05 and alive_pval > 0.05:
        stat, pval = ttest_ind(death_data[var], alive_data[var], equal_var=False)
        #test_type = 'Unpaired t-test'
        ttest_results.append((var,round(pval,4)))
    else:
        stat, pval = mannwhitneyu(death_data[var], alive_data[var], alternative='two-sided')
        #test_type = 'Mann-Whitney U test'
        mann_test_results.append((var,round(pval,4)))

    # Record the test results
    #test_results = test_results.append({'Variable': var, 'Test': test_type, 'Statistic': stat, 'p-value': pval},
                                       #ignore_index=True)

#print(test_results)
print("cate vars are: ",cat_vars)
print("num vars are ",numerical_vars)
print(chi_result)
print("=====================")
print(shapiro_test_results)
print("========================================")
print(mann_test_results)

print("========================================")
print(ttest_results)
print("--------------------")

output_dict= {
        'mann_whitney': mann_test_results,
        'ttest': ttest_results,
        'chi_square': chi_result,
        'shapiro_wilk': shapiro_test_results
    }


cate vars are:  ['amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin', 'MRA', 'heart_failure', 'AOS']
num vars are  ['SBP', 'DBP', 'PLT', 'LDL', 'HDL', 'LVEF', 'Na+', 'K+', 'MPV']
[('amiodarone', 0.0), ('loop_diuretics', 0.0), ('ivabradine', 0.0144), ('ARB', 0.2338), ('digoxin', 0.5185), ('MRA', 0.2884), ('heart_failure', 0.0), ('AOS', 0.974)]
[('SBP', (0.2579, 0.5882)), ('DBP', (0.5279, 0.3715)), ('PLT', (0.2361, 0.6935)), ('LDL', (0.9368, 0.95)), ('HDL', (0.4388, 0.0006)), ('LVEF', (0.0, 0.0)), ('Na+', (0.0, 0.0071)), ('K+', (0.0, 0.1771)), ('MPV', (0.0, 0.0003))]
[('HDL', 0.0), ('LVEF', 0.0), ('Na+', 0.4286), ('K+', 0.8569), ('MPV', 0.8662)]
[('SBP', 0.0), ('DBP', 0.0), ('PLT', 0.4739), ('LDL', 0.8392)]
--------------------


In [113]:
len(cat_vars)

8

In [114]:
len(numerical_vars)

9

In [115]:
df.columns

Index(['death', 'amiodarone', 'loop_diuretics', 'ivabradine', 'ARB', 'digoxin',
       'MRA', 'heart_failure', 'AOS', 'SBP', 'DBP', 'PLT', 'LDL', 'HDL',
       'LVEF', 'Na+', 'K+', 'MPV'],
      dtype='object')

In [116]:
dtype

NameError: name 'dtype' is not defined